In [1]:
!pip install dask[dataframe] --upgrade --quiet
!pip install dask-ml[complete] --quiet
!pip install aiohttp --quiet
!pip install joblib --quiet
!pip install dask distributed --upgrade --quiet
!pip install -U ipykernel --quiet
!pip install scikit-learn==0.23.2 --quiet

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, progress
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.metrics import roc_auc_score
import joblib
from dask_ml.model_selection import train_test_split
import pandas as pd
import warnings
import dask
import distributed

warnings.filterwarnings("ignore")

In [3]:
client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:39849 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


In [4]:
dask.__version__

'2021.01.0'

In [5]:
distributed.__version__

'2021.01.0'

In [6]:
# This loads the data into a Dask DataFrame
df = dd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/creditcard.csv', dtype={'Time': 'float64'})

df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [7]:
df.describe().compute()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.175161e-15,3.384974e-16,-1.341216e-15,2.088465e-15,9.707851e-16,1.494498e-15,-5.652268e-16,1.143626e-16,-2.409599e-15,2.236957e-15,1.679714e-15,-1.245415e-15,8.206966e-16,1.200708e-15,4.885859e-15,1.437017e-15,-3.784146e-16,9.596083e-16,1.037048e-15,6.402711e-16,1.640595e-16,-3.544643e-16,2.610582e-16,4.473116e-15,5.205196e-16,1.687298e-15,-3.666889e-16,-1.219469e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,1.088850e+00,1.020713e+00,9.992014e-01,9.952742e-01,9.585956e-01,9.153160e-01,8.762529e-01,8.493371e-01,8.381762e-01,8.140405e-01,7.709250e-01,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,-2.458826e+01,-4.797473e+00,-1.868371e+01,-5.791881e+00,-1.921433e+01,-4.498945e+00,-1.412985e+01,-2.516280e+01,-9.498746e+00,-7.213527e+00,-5.449772e+01,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,49346.000000,-7.868600e-01,-5.536365e-01,-7.156553e-01,-7.067841e-01,-4.093779e-01,-6.546632e-01,-4.787135e-01,-1.339191e-01,-5.477089e-01,-4.837668e-01,-5.417076e-01,-2.351421e-01,-6.201901e-01,-3.439514e-01,-3.409833e-01,-4.198368e-01,-4.071982e-01,-4.129756e-01,-3.581993e-01,-1.686500e-01,-2.256367e-01,-5.250530e-01,-1.301482e-01,-3.241623e-01,-2.218416e-01,-2.821043e-01,-6.089022e-02,-2.762231e-02,6.840000,0.000000
50%,76029.000000,9.301532e-02,8.999754e-02,1.743346e-01,1.809941e-01,1.480477e-01,-1.649056e-01,1.590763e-01,7.790153e-02,2.583745e-02,-9.153580e-02,1.428546e-01,2.220066e-01,5.421844e-03,6.496143e-02,3.185520e-01,8.990400e-02,-1.179887e-02,8.393824e-02,3.469481e-02,-2.513490e-02,6.636310e-03,1.241855e-01,5.160336e-02,6.794774e-02,1.674311e-01,-1.290092e-03,1.088230e-02,2.345040e-02,24.990000,0.000000
75%,138472.000000,1.912886e+00,8.916958e-01,1.373682e+00,1.012508e+00,8.613822e-01,4.775488e-01,7.334868e-01,3.706835e-01,7.092123e-01,4.814288e-01,1.041840e+00,6.508526e-01,6.986651e-01,5.466114e-01,8.783822e-01,5.405436e-01,4.455337e-01,5.905041e-01,4.781289e-01,1.678982e-01,2.385322e-01,7.308589e-01,2.346538e-01,5.274699e-01,4.190202e-01,2.938470e-01,1.060637e-01,8.061929e-02,84.910000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,2.374514e+01,1.201891e+01,7.848392e+00,7.126883e+00,1.052677e+01,8.877742e+00,1.731511e+01,9.253526e+00,5.041069e+00,5.591971e+00,3.942090e+01,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [8]:
# This is the feature set
X = df[["V1", "V2", "V3", "Amount"]]

# This is the target variable
Y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# Because your data can fit into memory,
# persist it to the RAM
X_train.persist()
X_test.persist()
y_train.persist()
y_test.persist()

Dask Series Structure:
npartitions=3
    int64
      ...
      ...
      ...
Name: Class, dtype: int64
Dask Name: split, 3 tasks

In [9]:
rf_model = RandomForestClassifier()

with joblib.parallel_backend('dask'):
    scores = cross_validate(rf_model, X_train.compute(), y_train.compute(), cv=4)
    
scores

{'fit_time': array([21.85528779, 56.67017007, 57.06225014, 54.48151708]),
 'score_time': array([0.12155294, 0.21826673, 0.2184217 , 0.21860576]),
 'test_score': array([0.99756123, 0.99835073, 0.99875428, 0.99854373])}

In [10]:
# Random forest classifier
rf_params = {"max_depth": [2, 4, 8, 16]}

rf_model = RandomForestClassifier()

grid_search_rf = GridSearchCV(rf_model,
                           param_grid=rf_params,
                           return_train_score=True,
                           iid=True,
                           cv=4,
                           n_jobs=-1, 
                           scoring='roc_auc')

In [11]:
with joblib.parallel_backend('dask'):
    grid_search_rf.fit(X_train.compute(), y_train.compute())

In [12]:
print("The best value is: ", grid_search_rf.best_params_)
print("The test AUC score is: ", grid_search_rf.score(X_test.compute(), y_test.compute()))

The best value is:  {'max_depth': 8}
The test AUC score is:  0.919151789203928


In [13]:
from dask_ml.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train.values.compute(), y_train.values.compute())

LogisticRegression()

In [14]:
preds_train = lr.predict(X_train.values.compute())
preds_test = lr.predict(X_test.values.compute())

print("Training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Training score is:  0.8433109683308756
Test score is:  0.7108254333694475


## In this task, you'll train several machine-learning models from scikit-learn, using Dask as the backend of joblib. This time, you need to use all of the variables except Class as your feature set. The Class variable will be your target variable.

In [15]:
# use all features, but drop target
X = df.drop('Class',axis=1)

# This is the target variable
y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Because your data can fit into memory,
# persist it to the RAM
X_train.persist()
X_test.persist()
y_train.persist()
y_test.persist()

Dask Series Structure:
npartitions=3
    int64
      ...
      ...
      ...
Name: Class, dtype: int64
Dask Name: split, 3 tasks

In [18]:
lr = LogisticRegression(fit_intercept=False)

with joblib.parallel_backend('dask'):
    lr.fit(X_train.compute(), y_train.compute())
    
preds_train = lr.predict(X_train.values.compute())
preds_test = lr.predict(X_test.values.compute())

print("Logistic regression training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Logistic regression test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Logistic regression training score is:  0.8338801703406193
Logistic regression test score is:  0.831027723834703


In [19]:
rfc = RandomForestClassifier()

with joblib.parallel_backend('dask'):
    rfc.fit(X_train.compute(), y_train.compute())
    
preds_train = rfc.predict(X_train.values.compute())
preds_test = rfc.predict(X_test.values.compute())

print("Random forest training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Random forest test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Random forest training score is:  1.0
Random forest test score is:  0.9850928681501826


In [20]:
gbc = GradientBoostingClassifier()

with joblib.parallel_backend('dask'):
    gbc.fit(X_train.compute(), y_train.compute())
    
preds_train = gbc.predict(X_train.values.compute())
preds_test = gbc.predict(X_test.values.compute())

print("Gradient boosting tree training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Gradient boosting tree test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Gradient boosting tree training score is:  0.9447045595837774
Gradient boosting tree test score is:  0.9239536850701433


## Compare the results of your models.

The random forest model performs best on the test data, even tough it is over-fitting. Logistic Regression performs the worst of the 3 models. 

It would be interesting to see how these models do with hyperparameter tuning. We used the default parameters for this project. In the future it would be nice to see which tuned model does best. 